In [1]:
import os
import re
import copy
import pandas as pd
import warnings

# DType warning when importing .csv files with parsed PMIDs. Avoid showing the warning
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)

In [2]:
# Directory containing .txt files containing PMIDs
PMID_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\


In [3]:
# Directory containing .csv files with parsed articles from XMLs
parsedDFs_input = input().strip()

 E:\sciAbstractsProject_parsedDFs\


In [4]:
# Combine all PMIDs in a single list
with open(PMID_input+"pmids_until_19991231.txt") as file_pmids_until_19991231:
    articles_pmids = [line.rstrip() for line in file_pmids_until_19991231]

with open(PMID_input+"pmids_20000101_until_20091231.txt") as file_pmids_20000101_until_20091231:
    articles_pmids_20000101_until_20091231 = [line.rstrip() for line in file_pmids_20000101_until_20091231]

with open(PMID_input+"pmids_20100101_until_20191231.txt") as file_pmids_20100101_until_20191231:
    articles_pmids_20100101_until_20191231 = [line.rstrip() for line in file_pmids_20100101_until_20191231]

with open(PMID_input+"pmids_20200101_until_20240701.txt") as file_pmids_20200101_until_20240701:
    articles_pmids_20200101_until_20240701 = [line.rstrip() for line in file_pmids_20200101_until_20240701]


articles_pmids.extend(articles_pmids_20000101_until_20091231)
articles_pmids.extend(articles_pmids_20100101_until_20191231)
articles_pmids.extend(articles_pmids_20200101_until_20240701)

In [5]:
# Number of PMIDs
len(articles_pmids)

37845132

In [6]:
# Separate in unique and duplicates PMIDs
unique_pmids = set()
duplicated_pmids = []

for pmid in articles_pmids:
    if pmid in unique_pmids:
        duplicated_pmids.append(pmid)
    else:
        unique_pmids.add(pmid)

In [7]:
# Number of unique PMIDs
len(unique_pmids)

37544208

In [8]:
# Number of duplicated PMIDs
len(duplicated_pmids)

300924

In [9]:
# Save lists of unique and duplicated PMIDs as .txt files
with open(os.path.join("E:", "PMIDsInitialParsing", "unique_PMIDs.txt"), 'w') as output:
    for row in unique_pmids:
        output.write(str(row) + '\n')

with open(os.path.join("E:", "PMIDsInitialParsing", "duplicated_PMIDs.txt"), 'w') as output:
    for row in duplicated_pmids:
        output.write(str(row) + '\n')

In [10]:
# Directories containing .csv files with parsed articles
os.listdir(parsedDFs_input)

['20000101_until_20091231',
 '20100101_until_20191231',
 '20200101_until_20240701',
 '_until_19991231']

In [11]:
# Extract and add names of files in these directories to a list
raw_parsedDFs = os.listdir(parsedDFs_input+'_until_19991231')
raw_parsedDFs_2000s = os.listdir(parsedDFs_input+'20000101_until_20091231')
raw_parsedDFs_2010s = os.listdir(parsedDFs_input+'20100101_until_20191231')
raw_parsedDFs_2020s = os.listdir(parsedDFs_input+'20200101_until_20240701')

for el in range(len(raw_parsedDFs)):
    raw_parsedDFs[el] = os.path.join('_until_19991231', raw_parsedDFs[el])

for el in range(len(raw_parsedDFs_2000s)):
    raw_parsedDFs_2000s[el] = os.path.join('20000101_until_20091231', raw_parsedDFs_2000s[el])

for el in range(len(raw_parsedDFs_2010s)):
    raw_parsedDFs_2010s[el] = os.path.join('20100101_until_20191231', raw_parsedDFs_2010s[el])

for el in range(len(raw_parsedDFs_2020s)):
    raw_parsedDFs_2020s[el] = os.path.join('20200101_until_20240701', raw_parsedDFs_2020s[el])

raw_parsedDFs.extend(raw_parsedDFs_2000s)
raw_parsedDFs.extend(raw_parsedDFs_2010s)
raw_parsedDFs.extend(raw_parsedDFs_2020s)

In [12]:
# Keep only .csv files
parsedDFs = []
for DF in raw_parsedDFs:
    if DF[-4:] == ".csv":
        parsedDFs.append(DF)

In [14]:
# For each .csv file, extract the values of 'PMID' column and add to a list
parsedDFs_PMIDs = []
for DF in parsedDFs:
    print("Extracting PMIDs from file: " + str(parsedDFs.index(DF)) + "/" + str(len(parsedDFs)))
    df = pd.read_csv(parsedDFs_input+DF)
    parsedDFs_PMIDs.extend(list(df["PMID"].values))

Extracting PMIDs from file: 0/381
Extracting PMIDs from file: 1/381
Extracting PMIDs from file: 2/381
Extracting PMIDs from file: 3/381
Extracting PMIDs from file: 4/381
Extracting PMIDs from file: 5/381
Extracting PMIDs from file: 6/381
Extracting PMIDs from file: 7/381
Extracting PMIDs from file: 8/381
Extracting PMIDs from file: 9/381
Extracting PMIDs from file: 10/381
Extracting PMIDs from file: 11/381
Extracting PMIDs from file: 12/381
Extracting PMIDs from file: 13/381
Extracting PMIDs from file: 14/381
Extracting PMIDs from file: 15/381
Extracting PMIDs from file: 16/381
Extracting PMIDs from file: 17/381
Extracting PMIDs from file: 18/381
Extracting PMIDs from file: 19/381
Extracting PMIDs from file: 20/381
Extracting PMIDs from file: 21/381
Extracting PMIDs from file: 22/381
Extracting PMIDs from file: 23/381
Extracting PMIDs from file: 24/381
Extracting PMIDs from file: 25/381
Extracting PMIDs from file: 26/381
Extracting PMIDs from file: 27/381
Extracting PMIDs from file: 28

In [45]:
# Separate in unique and duplicates parsed PMIDs
unique_pmids_parsed_DFs = set()
duplicated_pmids_parsed_DFs = []

for parsed_pmid in parsedDFs_PMIDs:
    if str(parsed_pmid) in unique_pmids_parsed_DFs:
        duplicated_pmids_parsed_DFs.append(str(parsed_pmid))
    else:
        unique_pmids_parsed_DFs.add(str(parsed_pmid))

In [46]:
len(parsedDFs_PMIDs)

37836648

In [47]:
len(unique_pmids_parsed_DFs)

37535754

In [48]:
len(duplicated_pmids_parsed_DFs)

300894

In [49]:
# Save lists of unique and duplicated parsed PMIDs as .txt files
with open(os.path.join("E:", "PMIDsInitialParsing", "parsed_unique_PMIDs.txt"), 'w') as output:
    for row in unique_pmids_parsed_DFs:
        output.write(str(row) + '\n')

with open(os.path.join("E:", "PMIDsInitialParsing", "parsed_duplicated_PMIDs.txt"), 'w') as output:
    for row in duplicated_pmids_parsed_DFs:
        output.write(str(row) + '\n')

In [50]:
# Subtract unique PMIDs - unique parsed PMIDs to get missing (not parsed) PMIDs
notParsedPMIDs = unique_pmids.difference(unique_pmids_parsed_DFs)

In [51]:
# Save missing (not parsed) PMIDs
with open(os.path.join("E:", "PMIDsInitialParsing", "not_parsed_PMIDS.txt"), 'w') as output:
    for row in notParsedPMIDs:
        output.write(str(row) + '\n')